In [2]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
data = []
count = 1
for year in range(1980,2020):
    website_url = requests.get('https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_per_game.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('table',{'id':"per_game_stats"})
    tabledata = My_table.findAll('td')
    for cell in tabledata:
        count = count+1
        if (count%29 == 2):
            data.append(str(year))
        data.append(cell.get_text())

In [4]:
import numpy as np
data2 = np.array(data)
refined = np.reshape(data2, (-1, 30))

In [5]:
tableheader = My_table.findAll('th')
headers = []
count = 0
for item in tableheader:
    if (count < 30):
        headers.append(item.get_text())
        count=count+1
headers.remove('Rk')
headers = ['Year'] + headers
print(headers)

['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']


In [6]:
import pandas as pd
df1 = pd.DataFrame(refined, columns=headers)
df1.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1980,Kareem Abdul-Jabbar*,C,32,LAL,82,,38.3,10.2,16.9,...,.765,2.3,8.5,10.8,4.5,1.0,3.4,3.6,2.6,24.8
1,1980,Tom Abernethy,PF,25,GSW,67,,18.2,2.3,4.7,...,.683,0.9,1.9,2.9,1.3,0.5,0.2,0.6,1.8,5.4
2,1980,Alvan Adams,C,25,PHO,75,,28.9,6.2,11.7,...,.797,2.1,6.0,8.1,4.3,1.4,0.7,2.9,3.2,14.9
3,1980,Tiny Archibald*,PG,31,BOS,80,80,35.8,4.8,9.9,...,.830,0.7,1.7,2.5,8.4,1.3,0.1,3.0,2.7,14.1
4,1980,Dennis Awtrey,C,31,CHI,26,,21.5,1.0,2.3,...,.640,1.1,3.3,4.4,1.5,0.5,0.6,1.0,2.5,3.3


In [7]:
df1 = df1.drop_duplicates(subset='Player', keep='first')
rookies = df1[df1['Year'] != '1980']
rookies.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
362,1981,Darrell Allums,PF,22,DAL,22,,12.5,1.0,3.0,...,.591,0.9,2.1,3.0,1.1,0.2,0.4,1.0,2.3,2.7
369,1981,Mel Bennett,PF,26,UTA,28,,11.2,0.9,2.1,...,.654,1.2,2.1,3.3,0.5,0.1,0.4,1.1,2.0,3.8
374,1981,Norman Black,SG,23,DET,3,,9.3,1.0,3.3,...,.250,0.0,0.7,0.7,0.7,0.3,0.0,0.3,0.7,2.7
385,1981,Dave Britton,PG,22,WSB,2,,4.5,1.0,1.5,...,,0.0,1.0,1.0,1.5,0.5,0.0,1.0,1.0,2.0
386,1981,Michael Brooks,SF,22,SDC,82,,30.2,6.0,12.4,...,.706,2.6,2.8,5.4,2.5,1.2,0.4,2.0,2.9,14.7


In [8]:
rookies.columns

Index(['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [9]:
rookies2 = rookies.set_index('Player')

In [10]:
rookies3 = rookies2.drop(['Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P','3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB','DRB', 'TOV', 'PF'], axis=1)
rookies3.head()

,Year,Pos,Age,TRB,AST,STL,BLK,PTS
Player,,,,,,,,
Darrell Allums,1981,PF,22,3.0,1.1,0.2,0.4,2.7
Mel Bennett,1981,PF,26,3.3,0.5,0.1,0.4,3.8
Norman Black,1981,SG,23,0.7,0.7,0.3,0.0,2.7
Dave Britton,1981,PG,22,1.0,1.5,0.5,0.0,2.0
Michael Brooks,1981,SF,22,5.4,2.5,1.2,0.4,14.7


In [11]:
rookies3['Age'] = pd.to_numeric(rookies2['Age'], errors='coerce')
rookies3['Year'] = pd.to_numeric(rookies2['Year'], errors='coerce')

In [12]:
rookies3.head()

,Year,Pos,Age,TRB,AST,STL,BLK,PTS
Player,,,,,,,,
Darrell Allums,1981,PF,22,3.0,1.1,0.2,0.4,2.7
Mel Bennett,1981,PF,26,3.3,0.5,0.1,0.4,3.8
Norman Black,1981,SG,23,0.7,0.7,0.3,0.0,2.7
Dave Britton,1981,PG,22,1.0,1.5,0.5,0.0,2.0
Michael Brooks,1981,SF,22,5.4,2.5,1.2,0.4,14.7


In [13]:
rookies3['NBATRB'] = rookies3['TRB']
rookies3['NBAAST'] = rookies3['AST']
rookies3['NBASTL'] = rookies3['STL']
rookies3['NBABLK'] = rookies3['BLK']
rookies3['NBAPTS'] = rookies3['PTS']
rookies4 = rookies3.drop(['TRB', 'AST', 'STL', 'BLK', 'PTS'], axis=1)
rookies4.head()

,Year,Pos,Age,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
Player,,,,,,,,
Darrell Allums,1981,PF,22,3.0,1.1,0.2,0.4,2.7
Mel Bennett,1981,PF,26,3.3,0.5,0.1,0.4,3.8
Norman Black,1981,SG,23,0.7,0.7,0.3,0.0,2.7
Dave Britton,1981,PG,22,1.0,1.5,0.5,0.0,2.0
Michael Brooks,1981,SF,22,5.4,2.5,1.2,0.4,14.7


In [14]:
players = rookies4.index.values

In [22]:
print(len(players))

2808


In [27]:
maindata = []

for name in players:
    realname = name
    name = name.lower()
    name = name.replace(" ", "-")
    website_url = requests.get('https://www.sports-reference.com/cbb/players/'+ name +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('table',{'id':'players_per_game'})
    if My_table is not None:
        maindata = maindata + [realname]
        tabledata = My_table.findAll('td')
        data = []
        for cell in tabledata:
            data.append(cell.get_text())
        maindata = maindata +data[-28:]
    print (name)

darrell-allums
mel-bennett
norman-black
dave-britton
michael-brooks
lewis-brown
rickey-brown
joe-barry-carroll
butch-carter
reggie-carter
art-collins
don-collins
darwin-cook
geoff-crompton
earl-cureton
monti-davis
ron-davis
james-donaldson
jacky-dorsey
larry-drew
ralph-drollinger
john-duren
tony-fuller
calvin-garrett
mike-gminski
rickey-green
darrell-griffith
bill-hanzlik
alan-hardy
mike-harper
keith-herron
kenny-higgs
cedrick-hordges
tony-jackson
lee-johnson
reggie-johnson
edgar-jones
walter-jordan
clarence-kea
chad-kinch
wayne-kreklow
bill-laimbeer
edmund-lawrence
ronnie-lester
scott-lloyd
kyle-macy
rick-mahorn
wes-matthews
bill-mayfield
andre-mccarter
keith-mccord
kevin-mchale*
dick-miller
johnny-moore
lowes-moore
kenny-natt
carl-nicks
mike-niles
mike-o'koren
jawann-oldham
louis-orr
myles-patrick
tony-price
wally-rank
kelvin-ransey
james-ray
billy-reid
wayne-robinson
lorenzo-romar
frankie-sanders
dewayne-scales
craig-shelton
jerry-sichting
larry-smith
rory-sparrow
john-stroud
carlos

jay-edwards
sean-elliott
pervis-ellison
derrick-gervin
greg-grant
scott-haffner
tom-hammonds
tim-hardaway
mike-higgins
ed-horton
byron-irvin
jaren-jackson
eric-johnson
nate-johnston
shawn-kemp
stan-kimbrough
stacey-king
frank-kornet
jeff-lebo
tim-legler
gary-leonard
clifford-lett
todd-lichti
roy-marble
šarūnas-marčiulionis*
jeff-martin
anthony-mason
bob-mccann
mel-mccants
george-mccloud
carlton-mckinney
scott-meents
sam-mitchell
mike-morrison
john-morton
dyron-nix
žarko-paspalj
kenny-payne
dražen-petrović*
brian-quinnett
j.r.-reid
glen-rice
pooh-richardson
clifford-robinson
david-robinson*
doug-roth
donald-royal
delaney-rudd
jeff-sanders
dexter-shouse
michael-smith
jay-taylor
leonard-taylor
henry-turner
gary-voce
alexander-volkov
doug-west
randy-white
mike-williams
haywoode-workman
alaa-abdelnaby
mahmoud-abdul-rauf
keith-askins
miloš-babić
cedric-ball
lance-blanks
anthony-bonner
dee-brown
jud-buechler
matt-bullard
willie-burton
rick-calloway
elden-campbell
duane-causwell
cedric-ceballo

mike-bibby
earl-boykins
gerald-brown
cory-carr
vince-carter
keon-clark
kornél-dávid
ricky-davis
michael-dickerson
michael-doleac
bryce-drew
marlon-garnett
pat-garrity
paul-grant
matt-harpring
al-harrington
j.r.-henderson
larry-hughes
randell-jackson
sam-jacobson
jerome-james
antawn-jamison
damon-jones
jonathan-kerner
gerard-king
raef-lafrentz
rashard-lewis
felipe-lópez
tyronn-lue
sean-marks
kelly-mccarty
jelani-mccoy
roshown-mcleod
brad-miller
cuttino-mobley
nazr-mohammed
mikki-moore
makhtar-n'diaye
tyrone-nesby
rasho-nesterović
dirk-nowitzki
michael-olowokandi
andrae-patterson
ruben-patterson
paul-pierce
casey-shaw
jeff-sheppard
miles-simon
alvin-sims
brian-skinner
ryan-stack
vladimir-stepania
peja-stojaković
bruno-šundov
robert-traylor
bonzi-wells
tyson-wheeler
jahidi-white
jason-williams
shammond-williams
trevor-winter
korleone-young
rafer-alston
chucky-atkins
william-avery
jonathan-bender
calvin-booth
lazaro-borrell
cal-bowdler
ryan-bowen
ira-bowman
a.j.-bramlett
elton-brand
greg-b

adam-morrison
david-noel
steve-novak
patrick-o'bryant
kevinn-pinkney
leon-powe
roger-powell
chris-quinn
allan-ray
j.j.-redick
jeremy-richardson
sergio-rodríguez
rajon-rondo
brandon-roy
thabo-sefolosha
mouhamed-sene
cedric-simmons
uroš-slokar
craig-smith
steven-smith
vassilis-spanoulis
tyrus-thomas
p.j.-tucker
marcus-vinicius
james-white
justin-williams
marcus-williams
shawne-williams
shelden-williams
arron-afflalo
blake-ahearn
lance-allred
morris-almond
joel-anthony
marco-belinelli
corey-brewer
aaron-brooks
wilson-chandler
mike-conley
daequan-cook
javaris-crittenton
jermareo-davidson
glen-davis
guillermo-díaz
jared-dudley
kevin-durant
nick-fazekas
kyrylo-fesenko
thomas-gardner
marcin-gortat
aaron-gray
jeff-green
taurean-green
mike-harris
spencer-hawes
al-horford
yi-jianlian
coby-karl
carl-landry
keith-langford
stéphane-lasme
acie-law
ian-mahinmi
dominic-mcguire
josh-mcroberts
jamario-moon
juan-carlos-navarro
demetris-nichols
joakim-noah
oleksiy-pecherov
kosta-perović
kasib-powell
gabe-

justin-anderson
thanasis-antetokounmpo
keith-appling
nemanja-bjelica
devin-booker
anthony-brown
willie-cauley-stein
rakeem-christmas
coty-clarke
pat-connaughton
branden-dawson
bryce-dejean-jones
sam-dekker
duje-dukan
jarell-eddie
cristiano-felício
jerian-grant
montrezl-harrell
aaron-harrison
mario-hezonja
darrun-hilliard
rondae-hollis-jefferson
richaun-holmes
marcelo-huertas
josh-huestis
r.j.-hunter
damien-inglis
stanley-johnson
nikola-jokić
tyus-jones
frank-kaminsky
sasha-kaun
kevon-looney
trey-lyles
boban-marjanović
jarell-martin
t.j.-mcconnell
chris-mccullough
jordan-mcrae
salah-mejri
jordan-mickey
luis-montero
emmanuel-mudiay
xavier-munford
raul-neto
j.j.-o'brien
jahlil-okafor
kelly-oubre
lamar-patterson
cameron-payne
tibor-pleiß
bobby-portis
kristaps-porziņģis
norman-powell
willie-reed
josh-richardson
terry-rozier
d'angelo-russell
jonathon-simmons
alex-stepheson
edy-tavares
axel-toupane
karl-anthony-towns
myles-turner
rashad-vaughn
briante-weber
alan-williams
justise-winslow
chris

In [66]:
import numpy as np
maindata2 = np.array(maindata)
refined2 = np.reshape(maindata2, (-1, 29))

In [67]:
tableheader = My_table.findAll('th')
colheaders = []
count = 0
for item in tableheader:
    if (count < 29):
        colheaders.append(item.get_text())
        count=count+1
colheaders.remove('Season')
colheaders = ['Name'] + colheaders
print(colheaders)

['Name', 'School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '\xa0', 'SOS']


In [69]:
import pandas as pd
college = pd.DataFrame(refined2, columns=colheaders)
college.head()

,Name,School,Conf,G,GS,MP,FG,FGA,FG%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,,SOS
0,Darrell Allums,,,,,,4.1,,6.10,UCLA,...,.688,3.3,0.5,0.3,0.3,1.1,1.5,3.8,,5.29
1,Mel Bennett,1.0,,2.4,,3.3,10.1,,0.95,Pitt,...,.682,10.2,1.0,,2.4,,3.3,10.1,,0.95
2,Norman Black,1.3,1.2,0.4,,2.6,15.5,,-2.66,St. Joseph's,...,.667,8.7,1.4,1.1,0.4,,3.1,16.6,,-1.10
3,Dave Britton,,,,,,12.0,,2.90,Texas A&M,...,.687,3.1,,,,,,11.1,,3.57
4,Michael Brooks,2.1,,,,,24.1,,-1.31,La Salle,...,.700,12.0,1.7,,,,1.2,23.1,,-1.28


In [70]:
college2 = college.drop(['\xa0'], axis=1)

In [71]:
college = college2

In [72]:
col = college.columns
cols = []
for co in col:
    cols.append(co)
cols.remove('Name')
cols.remove('School')
cols.remove('Conf')
print(cols)

['G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SOS']


In [73]:
for col in cols:
    college[col] = pd.to_numeric(college[col], errors='coerce')

In [74]:
college = college.dropna()
college = college.drop(['Conf'], axis=1)
college.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SOS
570,Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,...,0.6,0.9,1.5,0.2,0.2,0.2,0.7,1.0,3.2,-5.93
918,Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,...,2.9,3.4,8.9,0.6,1.1,1.2,1.9,3.3,14.6,0.74
1044,James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,...,0.7,2.0,2.7,0.4,0.6,0.3,0.9,2.4,2.9,-7.38
1188,Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,...,0.3,3.6,3.1,3.4,0.8,0.2,2.9,1.4,5.2,-7.20
1244,Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,...,0.8,2.8,3.8,2.2,1.9,0.2,2.7,2.4,15.8,11.13


In [75]:
college.to_csv('CollegeStatLog2.csv')


In [76]:
college.reset_index(inplace=True)

In [77]:
college.head()

,index,Name,School,G,GS,MP,FG,FGA,FG%,2P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SOS
0,570,Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,...,0.6,0.9,1.5,0.2,0.2,0.2,0.7,1.0,3.2,-5.93
1,918,Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,...,2.9,3.4,8.9,0.6,1.1,1.2,1.9,3.3,14.6,0.74
2,1044,James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,...,0.7,2.0,2.7,0.4,0.6,0.3,0.9,2.4,2.9,-7.38
3,1188,Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,...,0.3,3.6,3.1,3.4,0.8,0.2,2.9,1.4,5.2,-7.20
4,1244,Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,...,0.8,2.8,3.8,2.2,1.9,0.2,2.7,2.4,15.8,11.13


In [78]:
final = college.merge(rookies4, left_on='Name', right_on='Player')
final = final.drop(['index'], axis=1)
final = final.set_index('Name')
final.head()

,School,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,PTS,SOS,Year,Pos,Age,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
Name,,,,,,,,,,,,,,,,,,,,,
Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,0.510,...,3.2,-5.93,1990,SF,22,0.4,0.3,0.2,0.2,1.3
Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,0.581,...,14.6,0.74,1996,PF,24,5.6,0.6,0.4,0.2,6.3
James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,0.469,...,2.9,-7.38,1998,SG,22,0.7,0.0,0.1,0.1,2.7
Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,0.403,...,5.2,-7.20,2000,SF,23,2.7,0.5,0.7,0.4,7.0
Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,0.523,...,15.8,11.13,2002,PG,20,2.8,3.7,1.5,0.2,10.9


In [79]:
cols = rookies4.columns
for col in cols:
    final[col] = pd.to_numeric(final[col], errors='coerce')
final.head()

,School,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,PTS,SOS,Year,Pos,Age,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
Name,,,,,,,,,,,,,,,,,,,,,
Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,0.510,...,3.2,-5.93,1990,NaN,22,0.4,0.3,0.2,0.2,1.3
Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,0.581,...,14.6,0.74,1996,NaN,24,5.6,0.6,0.4,0.2,6.3
James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,0.469,...,2.9,-7.38,1998,NaN,22,0.7,0.0,0.1,0.1,2.7
Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,0.403,...,5.2,-7.20,2000,NaN,23,2.7,0.5,0.7,0.4,7.0
Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,0.523,...,15.8,11.13,2002,NaN,20,2.8,3.7,1.5,0.2,10.9


In [80]:
final.to_csv('CollegeRookieStatLog.csv')

In [81]:
df1.head()


,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1980,Kareem Abdul-Jabbar*,C,32,LAL,82,,38.3,10.2,16.9,...,.765,2.3,8.5,10.8,4.5,1.0,3.4,3.6,2.6,24.8
1,1980,Tom Abernethy,PF,25,GSW,67,,18.2,2.3,4.7,...,.683,0.9,1.9,2.9,1.3,0.5,0.2,0.6,1.8,5.4
2,1980,Alvan Adams,C,25,PHO,75,,28.9,6.2,11.7,...,.797,2.1,6.0,8.1,4.3,1.4,0.7,2.9,3.2,14.9
3,1980,Tiny Archibald*,PG,31,BOS,80,80,35.8,4.8,9.9,...,.830,0.7,1.7,2.5,8.4,1.3,0.1,3.0,2.7,14.1
4,1980,Dennis Awtrey,C,31,CHI,26,,21.5,1.0,2.3,...,.640,1.1,3.3,4.4,1.5,0.5,0.6,1.0,2.5,3.3


In [82]:
df4 = pd.read_csv('CollegeRookieStatLog.csv')
df4.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,PTS,SOS,Year,Pos,Age,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
0,Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,...,3.2,-5.93,1990,NaN,22,0.4,0.3,0.2,0.2,1.3
1,Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,...,14.6,0.74,1996,NaN,24,5.6,0.6,0.4,0.2,6.3
2,James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,...,2.9,-7.38,1998,NaN,22,0.7,0.0,0.1,0.1,2.7
3,Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,...,5.2,-7.20,2000,NaN,23,2.7,0.5,0.7,0.4,7.0
4,Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,...,15.8,11.13,2002,NaN,20,2.8,3.7,1.5,0.2,10.9


In [83]:
df4.columns

Index(['Name', 'School', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA',
       '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SOS', 'Year', 'Pos', 'Age',
       'NBATRB', 'NBAAST', 'NBASTL', 'NBABLK', 'NBAPTS'],
      dtype='object')

In [87]:
df5 = df4.drop(['NBATRB', 'NBAAST', 'NBASTL', 'NBABLK', 'NBAPTS'], axis=1)
df5.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,AST,STL,BLK,TOV,PF,PTS,SOS,Year,Pos,Age
0,Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,...,0.2,0.2,0.2,0.7,1.0,3.2,-5.93,1990,NaN,22
1,Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,...,0.6,1.1,1.2,1.9,3.3,14.6,0.74,1996,NaN,24
2,James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,...,0.4,0.6,0.3,0.9,2.4,2.9,-7.38,1998,NaN,22
3,Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,...,3.4,0.8,0.2,2.9,1.4,5.2,-7.20,2000,NaN,23
4,Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,...,2.2,1.9,0.2,2.7,2.4,15.8,11.13,2002,NaN,20


In [92]:
final = df5.merge(rookies4, left_on='Name', right_on='Player')
#final = final.drop(['index'], axis=1)
final = final.set_index('Name')
final.head()

,School,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,Pos_x,Age_x,Year_y,Pos_y,Age_y,NBATRB,NBAAST,NBASTL,NBABLK,NBAPTS
Name,,,,,,,,,,,,,,,,,,,,,
Mike Higgins,Navy,60.0,2.0,9.0,1.1,2.3,0.464,0.9,1.7,0.510,...,NaN,22,1990,SF,22,0.4,0.3,0.2,0.2,1.3
Ashraf Amaya,Southern Illinois,128.0,120.0,32.2,5.3,9.4,0.567,5.2,9.0,0.581,...,NaN,24,1996,PF,24,5.6,0.6,0.4,0.2,6.3
James Cotton,Maryland-Eastern Shore,19.0,3.0,13.9,0.8,1.8,0.441,0.8,1.7,0.469,...,NaN,22,1998,SG,22,0.7,0.0,0.1,0.1,2.7
Eddie Robinson,Lamar,114.0,75.0,32.6,1.8,4.9,0.365,1.5,3.8,0.403,...,NaN,23,2000,SF,23,2.7,0.5,0.7,0.4,7.0
Gilbert Arenas,Arizona,70.0,33.0,30.5,5.6,12.1,0.466,4.1,7.9,0.523,...,NaN,20,2002,PG,20,2.8,3.7,1.5,0.2,10.9


In [93]:
final.to_csv('CollegeRookieStatLog2.csv')
